In [1]:
from IPython.display import clear_output
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from typing import List

## 모델 진행상황 결과 확인하는 라이브러리
from tqdm.notebook import tqdm

## Huggingface 라이브러리 받아오는 코드
!pip install transformers SentencePiece
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py
from transformers import DistilBertModel
from tokenization_kobert import KoBertTokenizer

## 기타 사용할 머신러닝 알고리즘
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/MyDrive/의현/018_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.zip'
val_path = '/content/drive/MyDrive/의현/018_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.zip'
!cp -r "$train_path" ./
!cp -r "$val_path" ./
drive.flush_and_unmount()

In [4]:
!unzip './감성대화말뭉치(최종데이터)_Training.zip'
!unzip './감성대화말뭉치(최종데이터)_Validation.zip'
clear_output()

In [5]:
import os
import shutil
from sys import platform
from glob import glob

train_data_path = './감성대화말뭉치(최종데이터)_Training.xlsx'
val_data_path = './감성대화말뭉치(최종데이터)_Validation.xlsx'

if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    train_data_path = os.path.join("dset", train_data_path)
    val_data_path = os.path.join("dset", val_data_path)

In [6]:
import numpy as np
import pandas as pd

train_dataset = pd.read_excel(train_data_path, index_col = 'Unnamed: 0')
val_dataset = pd.read_excel(val_data_path, index_col = 'Unnamed: 0')

In [7]:
train_dataset[['사람문장1', '사람문장2', '사람문장3']] = train_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
train_dataset['sentence'] = train_dataset[['사람문장1', '사람문장2', '사람문장3']].apply(lambda x: ' '.join(x), axis=1)
train_dataset = train_dataset.drop(['사람문장1', '사람문장2', '사람문장3', '시스템문장1', '시스템문장2', '시스템문장3'], axis=1)
train_dataset['sentiment'] = train_dataset[['감정_대분류', '감정_소분류']].apply(lambda x: ' '.join(x), axis=1)
train_dataset = train_dataset.drop(['감정_대분류', '감정_소분류'], axis=1)

In [8]:
val_dataset[['사람문장1', '사람문장2', '사람문장3']] = val_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
val_dataset['sentence'] = val_dataset[['사람문장1', '사람문장2', '사람문장3']].apply(lambda x: ' '.join(x), axis=1)
val_dataset = val_dataset.drop(['사람문장1', '사람문장2', '사람문장3', '시스템문장1', '시스템문장2', '시스템문장3'], axis=1)
val_dataset['sentiment'] = val_dataset[['감정_대분류', '감정_소분류']].apply(lambda x: ' '.join(x), axis=1)
val_dataset = val_dataset.drop(['감정_대분류', '감정_소분류'], axis=1)

In [9]:
X_train = train_dataset.drop('sentiment', axis = 1)
y_train = train_dataset['sentiment']
X_val = val_dataset.drop('sentiment', axis = 1)
y_val = val_dataset['sentiment']

In [10]:
le = preprocessing.LabelEncoder()

train_labels = le.fit_transform(y_train.values)

In [11]:
num_unique_labels = len(le.classes_)
print("독립적인 레이블의 개수:", num_unique_labels)

독립적인 레이블의 개수: 60


In [12]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-04
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
clear_output()

In [67]:
class Dataset_Generation:
    def __init__(self, df, labels, tokenizer, max_len):
        self.len = len(df)
        self.data = df
        self.tokenizer = tokenizer
        self.labels = labels
        self.max_len = max_len

    def __getitem__(self, idx):
        title = ", ".join(str(_) for _ in self.data.loc[idx, :].values)
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding = 'max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [68]:
train_dataset = X_train
test_dataset = X_val
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

test_labels = le.transform(y_val.values)
training_set = Dataset_Generation(train_dataset, train_labels, tokenizer, MAX_LEN)
testing_set = Dataset_Generation(test_dataset, test_labels, tokenizer, MAX_LEN)

TRAIN Dataset: (51630, 5)
TEST Dataset: (6641, 5)


In [15]:
first_item = training_set[0]

for key, value in first_item.items():
    print(f"{key}: {value}")

ids: tensor([   2, 4485,   46, 3312,   46, 4360, 6079,   46, 7476, 6873,   46, 7342,
        7178,   46, 5000, 6881, 7089,   46, 3803, 7086, 3466, 5002, 5002, 1363,
        7096, 3282, 5591,  258, 5112, 5330, 1406, 5782,   54, 1189, 1435, 4999,
        7794,  921, 1370, 6797,   54, 1409, 5931, 7835, 2426, 4217,  517, 6751,
        7328, 3149,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    

In [16]:
", ".join(str(_) for _ in train_dataset.loc[0, :].values)

'청년, 여성, 진로,취업,직장, 해당없음, 일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. '

In [17]:
decoded_text = tokenizer.decode(first_item['ids'])
print(decoded_text)

[CLS] 청년, 여성, 진로,취업,직장, 해당없음, 일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD

In [69]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [19]:
class classification_with_BERTmodel(torch.nn.Module):
    def __init__(self):
        super(classification_with_BERTmodel, self).__init__()
        self.l1 = DistilBertModel.from_pretrained('monologg/distilkobert')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, num_unique_labels)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [76]:
torch.cuda.empty_cache()

RuntimeError: ignored

In [20]:
model = classification_with_BERTmodel()
model.to(device)
clear_output()

In [21]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [22]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [23]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0), total = len(training_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 100 steps: {loss_step}")
            print(f"Training Accuracy per 100 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}%")

    return

In [24]:
for epoch in range(EPOCHS):
    train(epoch)

  0%|          | 0/1614 [00:00<?, ?it/s]

Training Loss per 100 steps: 4.0947394371032715
Training Accuracy per 100 steps: 0.0
Training Loss per 100 steps: 3.2634146882818413
Training Accuracy per 100 steps: 20.526348651348652
The Total Accuracy for Epoch 0: 26.064303699399574
Training Loss Epoch: 3.027980426873416
Training Accuracy Epoch: 26.064303699399574%


  0%|          | 0/1614 [00:00<?, ?it/s]

Training Loss per 100 steps: 1.9820060729980469
Training Accuracy per 100 steps: 43.75
Training Loss per 100 steps: 2.417321338639274
Training Accuracy per 100 steps: 39.44805194805195
The Total Accuracy for Epoch 1: 39.754018981212475
Training Loss Epoch: 2.402009084026757
Training Accuracy Epoch: 39.754018981212475%


  0%|          | 0/1614 [00:00<?, ?it/s]

...

Training Loss per 100 steps: 0.22849223017692566
Training Accuracy per 100 steps: 96.875
Training Loss per 100 steps: 0.07878279850037152
Training Accuracy per 100 steps: 97.56493506493507
The Total Accuracy for Epoch 47: 97.10052295177222
Training Loss Epoch: 0.09289981415150503
Training Accuracy Epoch: 97.10052295177222%


  0%|          | 0/1614 [00:00<?, ?it/s]

Training Loss per 100 steps: 0.02639983780682087
Training Accuracy per 100 steps: 100.0
Training Loss per 100 steps: 0.0852597511609431
Training Accuracy per 100 steps: 97.36201298701299
The Total Accuracy for Epoch 48: 97.19349215572342
Training Loss Epoch: 0.09097666590843988
Training Accuracy Epoch: 97.19349215572342%


  0%|          | 0/1614 [00:00<?, ?it/s]

Training Loss per 100 steps: 0.025711486116051674
Training Accuracy per 100 steps: 100.0
Training Loss per 100 steps: 0.08740148057423339
Training Accuracy per 100 steps: 97.23089410589411
The Total Accuracy for Epoch 49: 97.11214410226613
Training Loss Epoch: 0.09169479128699796
Training Accuracy Epoch: 97.11214410226613%


In [35]:
model

classification_with_BERTmodel(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-2): 3 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [72]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss = 0; nb_tr_steps = 0; nb_tr_examples = 0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0), total = len(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples


    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [73]:
acc = valid(model, testing_loader)
print("Validation Data Accuracy = %0.2f%%" % acc)

  0%|          | 0/208 [00:00<?, ?it/s]

Validation Loss Epoch: 3.5779860724623385
Validation Accuracy Epoch: 58.0033127541033
Validation Data Accuracy = 58.00%


In [77]:
output_model_file = 'text_BERTmodel.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)

print('All files saved')

All files saved


In [102]:
age = '청년'
성별 = '남성'
상황 = '일상 대화'
txt = '무려 1003일만에 한화이글스 4연승!'

text = ", ".join([age, 성별, 상황, txt])
text = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length= 512,
            padding = 'max_length',
            return_token_type_ids=True,
            truncation=True
        )

outputs = model(torch.tensor(text['input_ids']).long().to(device), torch.tensor(text['attention_mask']).long().to(device)).squeeze()
big_val, big_idx = torch.max(outputs.data, dim=-1)
print(le.inverse_transform(big_idx.cpu().numpy().reshape(-1)))

['기쁨 기쁨']


In [111]:
age = '청년'
성별 = '남성'
상황 = '슬픔'
txt = '요즘 너무 힘들어. 내가 할 수 있는 건 아무것도 없어.'

text = ", ".join([age, 성별, 상황, txt])
text = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length= 512,
            padding = 'max_length',
            return_token_type_ids=True,
            truncation=True
        )

outputs = model(torch.tensor(text['input_ids']).long().to(device), torch.tensor(text['attention_mask']).long().to(device)).squeeze()
big_val, big_idx = torch.max(outputs.data, dim=-1)
print(le.inverse_transform(big_idx.cpu().numpy().reshape(-1)))

['상처 버려진']


In [105]:
def calculate_accu_sentiment(big_idx, targets):
    under_10_mask = (big_idx < 10) & (targets < 10)
    over_10_mask = (big_idx >= 10) & (targets >= 10)

    under_10_correct = (big_idx[under_10_mask] == targets[under_10_mask]).sum().item()
    over_10_correct = (big_idx[over_10_mask] // 10 == targets[over_10_mask] // 10).sum().item()

    n_correct = under_10_correct + over_10_correct

    return n_correct

In [108]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss = 0; nb_tr_steps = 0; nb_tr_examples = 0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0), total = len(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accu_sentiment(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%100==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples


    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [109]:
acc = valid(model, testing_loader)
print("Validation Data Accuracy = %0.2f%%" % acc)

  0%|          | 0/208 [00:00<?, ?it/s]

Validation Loss Epoch: 3.5818510622932362
Validation Accuracy Epoch: 71.32961903327812
Validation Data Accuracy = 71.33%
